In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import glob
import os 
import pathlib

Mounted at /content/drive


In [ ]:
''' Function to extract only the slices belonging to the lungs '''

def slice_selection (image, start, end):
  sp = image.shape
  new = np.zeros((end-start+1, sp[1], sp[2]))
  new = new.astype(np.int16) #convert the numpy array in int (zeros creates float numbers)
  j = 0
  for i in range(start , end+1):
    new[j] = image[i]
    j = j+1
  return new

# Below are defined the initial and final slices of the lungs for each patient, both ADC and SCC
# Those initial and final slices have been selected "manually", this means that looking at the plot of the scans, 
# the initial and final slices of the lungs have been chosen. Thus, when using this code, you have to choose the 
# initial and final slices to maintain an write them in these vectors. The same also applies to the 4 limits to cut the black parts around the lung volume reported below.
starts = [105,105,390,380,430,475,480,530,515,440,500,450,480,445,415,520,435,620,600,475,465,420,430,560,600,460,440,440,485,450,475,555,590,500]
ends = [260,260,580,585,585,645,650,665,640,630,600,620,625,620,580,705,675,750,770,585,590,560,550,740,725,640,630,620,650,635,610,750,720,670]

# Below are defined the 4 limits to cut the black parts around the lung volume for the ADC patients
up = [228,228,228,198,238,228,221,245,214,265,237,207,197,250,210,253,273,255,180,253,258,258,264,230,239,265,275,245,258,258,272,220,245,208]
down = [508,508,465,485,474,513,505,494,488,505,511,454,490,504,455,504,477,470,450,500,495,510,510,510,489,529,529,519,487,487,482,529,518,516]
left = [135,135,163,149,143,129,117,166,143,173,144,129,158,90,160,140,165,115,145,160,160,151,157,155,155,146,151,98,99,154,160,112,145,108]
right = [564,564,529,525,519,568,557,538,567,528,570,570,550,510,514,563,514,500,540,547,547,563,552,565,562,585,565,599,602,524,503,579,563,590]

In [ ]:
''' INSERTING ALL THE SCANS IN A VECTOR '''

scans = []
path = '/content/drive/MyDrive/...' # add path of preprocessed scans
for r, d, f in os.walk(path):
  for file in f: 
    if file.endswith('.npy'):
      scans.append(os.path.join(r, file))

In [ ]:
''' SELECTION OF THE LUNGS AND CUT OF THE BLACK PART'''

i = 0
for i in range(len(scans)):
  scan = np.load(scans[i])
  slice_selection = slice_selection(scan, starts[i], ends[i])
  scan_cut = np.zeros([slice_selection.shape[0],down[i]-up[i],right[i]-left[i]], dtype = int)
  for j in range(slice_selection.shape[0]):
    scan_cut[j] = slice_selection[j][up[i]:down[i],left[i]:right[i]]
  scan_cut = scan_cut.astype(np.int16)
  np.save('/content/drive/MyDrive/.../only_lung-'+str(i+1), scan_cut) #insert the path where to save the only_lung scans